In [ ]:
import serial
import csv

# 設定串口參數
serial_port = 'COM4'  # 根據你的設備改變這個值
baud_rate = 115200
timeout = 2

# 建立串口連接
ser = serial.Serial(serial_port, baud_rate, timeout=timeout)

# 定義每個測量類型的CSV文件名稱
csv_files = {
    'micr_micr_array.csv',
    #'micr_micr.csv',
}

# 打開CSV文件以便寫入
file_handlers = {key: open(file, mode='w', newline='') for key, file in csv_files.items()}
csv_writers = {key: csv.writer(fh) for key, fh in file_handlers.items()}

for key in csv_writers:
    csv_writers[key].writerow([key])  # 寫入標題行

try:
    line_count = 0
    max_lines = 1000  # 設置最大接收行數

    while line_count < max_lines:
        # 從串口讀取一行數據
        line = ser.readline().decode('utf-8', errors='ignore').strip()
        if not line:
            continue

        print(f"接收到的行: {line}")  # 調試輸出接收到的數據行

        # 檢查和解析數據行
        for key in csv_writers.keys():
            if line.startswith(key + ':'):
                try:
                    error_value = float(line.split(':')[1].strip())
                    csv_writers[key].writerow([error_value])
                    print(f"更新 {key}: {error_value}")  # 調試輸出更新的數據
                    line_count += 1  # 增加計數器
                except (IndexError, ValueError) as e:
                    print(f"解析行時出錯: {line} ({e})")  # 調試輸出錯誤信息

except KeyboardInterrupt:
    pass  # 允許通過 Ctrl+C 退出

finally:
    ser.close()
    for fh in file_handlers.values():
        fh.close()
    print(f"資料收集已結束，共接收了 {line_count} 行數據。")
